In [1]:
import math
import numpy as np
import os
from pymbar import MBAR

top_dir = os.getcwd()

#===================================================================================================
# INPUTS
#===================================================================================================

K = 8.314472 * 0.001  # Gas constant in kJ/mol/K    kJ/mol/K
V = 1661              # average volume per molecule for a reference concentration of  is 1.661 x 10^3 Å^3/molecule.

T      = 300.0          # Temperature in Kelvin

r0      = 0       # Distance in A
thA_0   = 0     # Angle in degrees (protein)
thB_0   = 0     # Angle in degrees (ligand)

#  kcal * 4.186 ---->  kJ
#
# AMBER restraint = k * d_x ^ 2, boresch u = 1/2 * k * d_x ^ 2
# 
# BORESCH FORMULA
K_r    = 2 * 10 * 4.186      # force constant for distance (kJ/mol/A^2)  
K_thA  = 2 * 100 * 4.186     # force constant for angle    (kJ/mol/rad^2)
K_thB  = 2 * 100 * 4.186     # force constant for angle    (kJ/mol/rad^2)
K_phiA = 2 * 100 * 4.186     # force constant for dihedral (kJ/mol/rad^2)
K_phiB = 2 * 100 * 4.186     # force constant for dihedral (kJ/mol/rad^2)
K_phiC = 2 * 100 * 4.186     # force constant for dihedral (kJ/mol/rad^2)

# ROUX FORMULA
K_dist    = 2 * 10 * 4.186     # force constant for distance    (kJ/mol/A^2)  
K_ang     = 2 * 100 * 4.186    # force constant for angle       (kJ/mol/rad^2)
K_rotate  = 2 * 100 * 4.186    # force constant for dihedral    (kJ/mol/rad^2)

# AMBER RESTRAINT CONSTANT
K_DIST      = 10           # force constant for distance    (kcal/mol/A^2)  
K_ANG       = 100          # force constant for angle       (kcal/mol/rad^2)
K_ROTATE    = 100          # force constant for dihedral    (kcal/mol/rad^2)

In [2]:
# read in balanced values
with open('./init_structure/Boresch_restraint.tmpl','r') as f:
    lines = [line for line in f.readlines() if line.strip()]
    r0 = float(lines[1].split()[1].split('=')[1].strip(','))
    thA_0 = float(lines[5].split()[1].split('=')[1].strip(','))
    thB_0 = float(lines[9].split()[1].split('=')[1].strip(','))
    dihedral_P_0 = float(lines[13].split()[1].split('=')[1].strip(','))
    dihedral_M_0 = float(lines[17].split()[1].split('=')[1].strip(','))
    dihedral_L_0 = float(lines[21].split()[1].split('=')[1].strip(','))

In [3]:
#===================================================================================================
# BORESCH FORMULA
#===================================================================================================

thA = math.radians(thA_0)  # convert angle from degrees to radians --> math.sin() wants radians
thB = math.radians(thB_0)  # convert angle from degrees to radians --> math.sin() wants radians

arg =(
    (8.0 * math.pi**2.0 * V) / (r0**2.0 * math.sin(thA) * math.sin(thB)) 
    * 
    (
        ( (K_r * K_thA * K_thB * K_phiA * K_phiB * K_phiC)**0.5 ) / ( (2.0 * math.pi * K * T)**(3.0) )
    )
)

In [4]:
dG = - K * T * math.log(arg)

print ("dG_off = %8.3f kcal/mol" %(dG/4.186))
print ("dG_on  = %8.3f kcal/mol" %(-dG/4.186))
# 1 kcal = 4.186 KJ

dG_off =  -12.115 kcal/mol
dG_on  =   12.115 kcal/mol


In [5]:
#===================================================================================================
# ROUX FORMULA
#===================================================================================================

thA = math.radians(thA_0)  # convert angle from degrees to radians --> math.sin() wants radians
thB = math.radians(thB_0)  # convert angle from degrees to radians --> math.sin() wants radians

Ft = r0 ** 2 * math.sin(thA) * ((2.0 * math.pi * K * T)**(3.0) / (K_dist * K_ang ** 2)) ** 0.5

Fr = math.sin(thB) * (2.0 * math.pi * K * T / K_rotate) ** 1.5 / (8 * math.pi ** 2)

In [6]:
dG = - K * T * math.log(Ft / V * Fr)
print (f"dG = {dG/4.186:.3f} kcal/mol")

dG = 12.115 kcal/mol


In [7]:
data_dir = './complex_1_10_100_100/post_processing'
#restraints = ['Distance', 'Angle_P', 'Angle_L', 'Dihedral_P','Dihedral_M','Dihedral_L']
mbar_energy = []
with open(f'{data_dir}/fort.7','r') as f:
    lines = [line for line in f.readlines() if line.startswith(' NMR')]
    print(lines[0].split())
    for line in lines:
        energy = float(line.split()[4]) + float(line.split()[7]) + float(line.split()[10])
        mbar_energy.append(energy)
mbar_state_1_0 = np.array(mbar_energy)

['NMR', 'restraints:', 'Bond', '=', '3.051', 'Angle', '=', '22.480', 'Torsion', '=', '13.093']


In [8]:
states = [ 0.0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.35, 0.5, 0.75, 1.0]

In [9]:
beta = 1.0 / (300 * 1.380649e-23 )
# (RESTRAINT * 4.186 * 1000 / 6.02214076e23) * beta
u_kn = [mbar_state_1_0 * 4.186 * 1000 / 6.02214076e23 * beta * state for state in states]

In [10]:
mbar_energy = np.array(u_kn)

In [11]:
N_k = np.full(len(states),1000,dtype=np.int32)

In [12]:
from pymbar import MBAR

In [13]:
mbar = MBAR(u_kn,N_k)

In [14]:
Delta_f_ij, dDelta_f_ij, Theta_ij = mbar.getFreeEnergyDifferences(return_theta=True)

In [15]:
dDelta_f_ij

array([[0.        , 0.00513006, 0.0095153 , 0.01372058, 0.01632013,
        0.01818238, 0.02086369, 0.0228555 , 0.02709622, 0.0301374 ,
        0.03400275, 0.03714743],
       [0.00513006, 0.        , 0.00465145, 0.00930736, 0.0122631 ,
        0.0144038 , 0.01749016, 0.01976394, 0.02450854, 0.02783016,
        0.03197577, 0.0353019 ],
       [0.0095153 , 0.00465145, 0.        , 0.00490534, 0.00815032,
        0.01055413, 0.01406303, 0.01664878, 0.021965  , 0.02560184,
        0.03005329, 0.0335708 ],
       [0.01372058, 0.00930736, 0.00490534, 0.        , 0.0034231 ,
        0.00604992, 0.00998909, 0.01293234, 0.01896144, 0.02300864,
        0.02785744, 0.03161839],
       [0.01632013, 0.0122631 , 0.00815032, 0.0034231 , 0.        ,
        0.00270253, 0.00686313, 0.01003269, 0.01658721, 0.02096827,
        0.02615217, 0.03011948],
       [0.01818238, 0.0144038 , 0.01055413, 0.00604992, 0.00270253,
        0.        , 0.00425855, 0.0075683 , 0.0145177 , 0.01918183,
        0.02466625,

In [16]:
print('Delta_f_ij.shape:',Delta_f_ij.shape)
print(f"Restraint energy : {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol")
with open(f'{data_dir}/Restraint_energy.txt','w') as f:
    f.write(f'Restraint energy(MBAR): {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol\n')
    for i in range(11):
        f.writelines(f'{Delta_f_ij[i][i+1] * 0.596:.3f}\n')

with open(f'{top_dir}/summary_ABFE.txt','a') as f:
    f.write('Boresch formula analysis\n')
    f.write(f'\tdG_analysis = \t{dG/4.186:.3f} \tkcal/mol\n')
    f.write('remove restraint in complex\n')
    f.write(f'\t\t-{Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f}\t± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} \tKcal/mol\n')

Delta_f_ij.shape: (12, 12)
Restraint energy : 4.473 ± 0.022 Kcal/mol


In [17]:
import pandas as pd
pd.DataFrame(Delta_f_ij * 0.596)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,0.222224,0.493604,0.848898,1.132555,1.371670,1.765390,2.086161,2.808138,3.332807,3.983261,4.473542
1,-0.222224,0.000000,0.271380,0.626673,0.910331,1.149446,1.543166,1.863937,2.585913,3.110583,3.761037,4.251318
2,-0.493604,-0.271380,0.000000,0.355294,0.638951,0.878066,1.271786,1.592557,2.314534,2.839203,3.489657,3.979938
3,-0.848898,-0.626673,-0.355294,0.000000,0.283658,0.522772,0.916492,1.237263,1.959240,2.483909,3.134364,3.624645
4,-1.132555,-0.910331,-0.638951,-0.283658,0.000000,0.239115,0.632835,0.953606,1.675582,2.200252,2.850706,3.340987
5,-1.371670,-1.149446,-0.878066,-0.522772,-0.239115,0.000000,0.393720,0.714491,1.436468,1.961137,2.611591,3.101872
6,-1.765390,-1.543166,-1.271786,-0.916492,-0.632835,-0.393720,0.000000,0.320771,1.042748,1.567417,2.217871,2.708152
7,-2.086161,-1.863937,-1.592557,-1.237263,-0.953606,-0.714491,-0.320771,0.000000,0.721977,1.246646,1.897100,2.387381
8,-2.808138,-2.585913,-2.314534,-1.959240,-1.675582,-1.436468,-1.042748,-0.721977,0.000000,0.524669,1.175123,1.665404
9,-3.332807,-3.110583,-2.839203,-2.483909,-2.200252,-1.961137,-1.567417,-1.246646,-0.524669,0.000000,0.650454,1.140735


In [18]:
O_ij = mbar.computeOverlap()['matrix']
np.savetxt(f'{data_dir}/restraint_overlap.txt',O_ij,fmt='%.8f')
O_ij.round(decimals=3, out=O_ij)
pd.DataFrame(O_ij)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.240,0.198,0.156,0.114,0.088,0.072,0.051,0.038,0.020,0.012,0.007,0.004
1,0.198,0.178,0.153,0.121,0.099,0.083,0.062,0.048,0.026,0.016,0.009,0.006
2,0.156,0.153,0.143,0.126,0.110,0.096,0.076,0.061,0.035,0.023,0.013,0.008
3,0.114,0.121,0.126,0.124,0.117,0.109,0.093,0.079,0.051,0.034,0.020,0.013
4,0.088,0.099,0.110,0.117,0.117,0.114,0.104,0.093,0.064,0.046,0.028,0.019
5,0.072,0.083,0.096,0.109,0.114,0.115,0.111,0.103,0.077,0.058,0.037,0.026
6,0.051,0.062,0.076,0.093,0.104,0.111,0.116,0.115,0.098,0.079,0.056,0.041
7,0.038,0.048,0.061,0.079,0.093,0.103,0.115,0.120,0.114,0.098,0.074,0.057
8,0.020,0.026,0.035,0.051,0.064,0.077,0.098,0.114,0.137,0.139,0.127,0.112
9,0.012,0.016,0.023,0.034,0.046,0.058,0.079,0.098,0.139,0.160,0.169,0.165
